In [1]:
%load_ext autoreload
%autoreload 1
import gensim
import pyspark.sql.functions as fn
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

%aimport ihop.community2vec
%aimport ihop.import_data

2022-01-06 20:11:11,440 : INFO : NumExpr defaulting to 4 threads.


In [2]:
spark = ihop.utils.get_spark_session("Community2vec Notebook")

22/01/06 20:11:17 WARN Utils: Your hostname, Kurt resolves to a loopback address: 127.0.1.1; using 192.168.0.11 instead (on interface wlp4s0)
22/01/06 20:11:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/06 20:11:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/06 20:11:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Spark configuration:
[('spark.driver.port', '39667'), ('spark.app.id', 'local-1641517883961'), ('spark.executor.id', 'driver'), ('spark.driver.host', '192.168.0.11'), ('spark.sql.warehouse.dir', 'file:/home/virginia/Documents/CenterForDataScience/ZuckermanProj/IHOP/notebooks/spark-warehouse'), ('spark.app.name', 'Community2vec Notebook'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.app.startTime', '1641517879610'), ('spark.executor.memory', '4G'), ('spark.executor.extraLibraryPath', '/home/virginia/hadoop-3.3.1/lib/native'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.extraLibraryPath', '/home/virginia/hadoop-3.3.1/lib/native')]


In [3]:
context_df = spark.read.csv("../data/community2vec/sample_data/user_contexts").select(fn.col("_c0").alias("subreddits"))
context_df.show()

+--------------------+
|          subreddits|
+--------------------+
|              cigars|
|          edc_raffle|
|             Calgary|
|              spacex|
|           AskReddit|
|            greenday|
|              boston|
|           pokemongo|
|         TheaterDays|
|       SquaredCircle|
|           Stellaris|
|     DBZDokkanBattle|
|The_Donald The_Do...|
|          Beekeeping|
|      AboveandBeyond|
|                 nba|
|       psychotherapy|
|        youtubehaiku|
|Anarcho_Capitalis...|
|                 PS4|
+--------------------+
only showing top 20 rows



In [4]:
# Grab window for skip-grams - needs to be the maximum number of comments a user had in a given month
# TODO Could try creating an iterator over the spark DF that gets passed to gensim, but let's read from the single bzip for now
df_split = context_df.select(fn.split("subreddits"," ").alias('subreddit_list')).select("subreddit_list", fn.size("subreddit_list").alias("num_comments"))
max_comments = df_split.agg(fn.max("num_comments")).head()[0]
print(max_comments)
df_split.show()

258
+--------------------+------------+
|      subreddit_list|num_comments|
+--------------------+------------+
|            [cigars]|           1|
|        [edc_raffle]|           1|
|           [Calgary]|           1|
|            [spacex]|           1|
|         [AskReddit]|           1|
|          [greenday]|           1|
|            [boston]|           1|
|         [pokemongo]|           1|
|       [TheaterDays]|           1|
|     [SquaredCircle]|           1|
|         [Stellaris]|           1|
|   [DBZDokkanBattle]|           1|
|[The_Donald, The_...|           2|
|        [Beekeeping]|           1|
|    [AboveandBeyond]|           1|
|               [nba]|           1|
|     [psychotherapy]|           1|
|      [youtubehaiku]|           1|
|[Anarcho_Capitali...|           2|
|               [PS4]|           1|
+--------------------+------------+
only showing top 20 rows



In [5]:
vocab = ihop.community2vec.get_vocabulary("../data/community2vec/sample_data/subreddit_counts.csv")
print("VOCAB SIZE:", len(vocab))
print("VOCAB SNIPPET:")
for i, (k,v) in enumerate(vocab.items()):
    print(k,":",v)
    if i>10:
        break

print("TOTAL COMMENT COUNT:", sum(vocab.values()))

VOCAB SIZE: 2275
VOCAB SNIPPET:
AskReddit : 486
CFB : 403
CrazyIdeas : 261
news : 158
ConciseIAmA : 147
4chan4trump : 136
politics : 117
RocketLeagueExchange : 96
The_Donald : 90
nba : 90
edc_raffle : 77
worldnews : 74
TOTAL COMMENT COUNT: 10000


In [6]:
num_users = context_df.count()
print(num_users)

7231


In [7]:
c2v_model = gensim.models.word2vec.Word2Vec(vector_size=150, min_count=0, window=max_comments, sg=1, hs=0, negative=35, sample=0)
c2v_model.build_vocab_from_freq(vocab)
print("Random initialization")
print(c2v_model.wv.most_similar("AskReddit", topn=10))
c2v_model.train(gensim.models.word2vec.PathLineSentences("../data/community2vec/sample_data/user_contexts/part-00000-0ad8096e-a7b4-4941-8946-0a8c7cb8108d-c000.csv.bz2"), total_examples=num_users, epochs=100, compute_loss=True)

2022-01-06 20:11:51,620 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=150, alpha=0.025)', 'datetime': '2022-01-06T20:11:51.620589', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 13:49:34) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.0-92-generic-x86_64-with-glibc2.17', 'event': 'created'}
2022-01-06 20:11:51,622 : INFO : Processing provided word frequencies
2022-01-06 20:11:51,623 : INFO : collected 2275 unique word types, with total frequency of 10000
2022-01-06 20:11:51,624 : INFO : Creating a fresh vocabulary
2022-01-06 20:11:51,677 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=0 retains 2275 unique words (100.0%% of original 2275, drops 0)', 'datetime': '2022-01-06T20:11:51.677487', 'gensim': '4.1.2', 'python': '3.8.12 (default, Oct 12 2021, 13:49:34) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.0-92-generic-x86_64-with-glibc2.17', 'event': 'prepare_vocab'}
2022-01-06 20:11:51,697 : INFO : Word2Vec lifecycle event {'msg': 'effective_

Random initialization
[('ImagesOfThe2010s', 0.26547014713287354), ('confession', 0.2560444176197052), ('circlejerk', 0.25592470169067383), ('FFXV', 0.23708847165107727), ('ImagesOfNetherlands', 0.22937066853046417), ('BannedFromThe_Donald', 0.22841127216815948), ('GundamExVs', 0.22638258337974548), ('neovim', 0.2199598252773285), ('Drugs', 0.21625253558158875), ('Anxiety', 0.21418383717536926)]


2022-01-06 20:11:52,109 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-01-06 20:11:52,111 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-01-06 20:11:52,994 : INFO : EPOCH 1 - PROGRESS: at 100.00% examples, 9125 words/s, in_qsize 0, out_qsize 1
2022-01-06 20:11:52,995 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-01-06 20:11:52,996 : INFO : EPOCH - 1 : training on 9221 raw words (9221 effective words) took 1.0s, 9108 effective words/s
2022-01-06 20:11:53,033 : INFO : reading file ../data/community2vec/sample_data/user_contexts/part-00000-0ad8096e-a7b4-4941-8946-0a8c7cb8108d-c000.csv.bz2
2022-01-06 20:11:53,135 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-01-06 20:11:53,138 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-01-06 20:11:53,906 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-01-06 20:11:53,907 : INFO : EPOCH - 2 : training on 9

(922100, 922100)

In [8]:
c2v_model.wv.most_similar("AskReddit", topn=10)

[('dirtykikpals', 0.9316475987434387),
 ('houston', 0.9131841659545898),
 ('soccer', 0.9104990363121033),
 ('lfg', 0.9103761315345764),
 ('AndroidQuestions', 0.909914493560791),
 ('progresspics', 0.9093375205993652),
 ('SmiteTrades', 0.9082111120223999),
 ('pic', 0.907174289226532),
 ('RandomActsOfBlowJob', 0.9071497321128845),
 ('relationships', 0.9071356058120728)]

In [9]:
c2v_model.wv.most_similar("news", topn=10)

[('vegan', 0.9960822463035583),
 ('science', 0.9955615997314453),
 ('democrats', 0.9939213395118713),
 ('nottheonion', 0.9722349047660828),
 ('business', 0.8650807738304138),
 ('MovieSuggestions', 0.8624818325042725),
 ('worldnews', 0.8343978524208069),
 ('imdbvg', 0.8080923557281494),
 ('Showerthoughts', 0.8064515590667725),
 ('raimimemes', 0.8061143755912781)]